<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_simplified_fixed_v3_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_simplified_fixed_v3.py

4-bit parity classification with a robust QNG workflow:

1) Analytic CNOT cascade baseline (100% accuracy)
2) Variational QNN baseline (5-fold CV)
3) Simplified quantum metric learning:
   - AngleEmbedding + baked-in parity CNOT ring
   - Single trainable Ry per qubit
   - QNGOptimizer with explicit metric tensor (argnum=[0])
   - Centered kernel–target alignment loss
4) Final SVM (precomputed kernel) 5-fold CV
5) Kernel heatmap visualization
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Hyperparameters
# -----------------------------------------------------------------------------
n_bits     = 4       # number of input qubits
qnn_epochs = 30      # epochs for variational QNN
qnn_lr     = 0.4     # learning rate for QNN
n_steps    = 50      # QNG training steps
qng_lr     = 0.1     # QNG step size
svm_C      = 1.0     # SVM regularization
seed       = 42

np.random.seed(seed)
pnp.random.seed(seed)

# -----------------------------------------------------------------------------
# 1) Dataset: 4-bit parity
# -----------------------------------------------------------------------------
X_int = pnp.array(list(itertools.product([0, 1], repeat=n_bits)), dtype=int)
y     = pnp.array(np.sum(X_int, axis=1) % 2, dtype=int)    # {0,1}
y_signed = 1 - 2 * y                                       # {+1,-1}

# For quantum feature map (AngleEmbedding expects floats)
X_feat = X_int.astype(float)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# -----------------------------------------------------------------------------
# 2) Analytic CNOT cascade baseline
# -----------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_analytic = [0 if analytic_circuit(x) > 0 else 1 for x in X_int]
print(f"Analytic CNOT Cascade Accuracy: {accuracy_score(y, preds_analytic):.3f}")

# -----------------------------------------------------------------------------
# 3) Variational QNN baseline (5-fold CV)
# -----------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

var_scores = []
for train_idx, test_idx in kf.split(X_int, y):
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(qnn_lr)

    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X_int[i]) for i in train_idx]
            return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
        params = opt.step(cost, params)

    preds_test = [qnn_circuit(params, X_int[i]) for i in test_idx]
    bits = [0 if v > 0 else 1 for v in preds_test]
    var_scores.append(accuracy_score(y[test_idx], bits))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# -----------------------------------------------------------------------------
# 4) Simplified quantum metric learning via QNG
# -----------------------------------------------------------------------------
dev_feat = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_feat, interface="autograd")
def feature_map_shallow(params, x):
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    for i in range(n_bits):
        qml.CNOT(wires=[i, (i + 1) % n_bits])
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    return qml.state()

def build_kernel_matrix(params):
    states = pnp.stack([feature_map_shallow(params, x) for x in X_feat])
    re, im = pnp.real(states), pnp.imag(states)
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(K):
    N  = K.shape[0]
    H  = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc = H @ K @ H
    T  = pnp.outer(y_signed, y_signed)
    return pnp.sum(Kc * T) / (pnp.linalg.norm(Kc) * pnp.linalg.norm(T))

params_k = pnp.random.randn(n_bits, requires_grad=True) * 0.1

# compute raw metric-tensor function (only params trainable)
raw_metric = qml.metric_tensor(feature_map_shallow, argnum=[0])

def metric_tensor_fn(params):
    mats = []
    for x in X_feat:
        m = raw_metric(params, x)
        # unpack if tuple
        if isinstance(m, tuple):
            m = m[0]
        mats.append(m)
    return sum(mats) / len(mats)

opt_qng = qml.QNGOptimizer(stepsize=qng_lr)

print("\nTraining simplified feature map via QNG + centered alignment:")
for step in range(n_steps):
    loss_fn = lambda p: -centered_alignment(build_kernel_matrix(p))

    params_k = opt_qng.step(
        loss_fn,
        params_k,
        metric_tensor_fn=metric_tensor_fn
    )

    if step % 10 == 0:
        align = centered_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:>3}: alignment = {align:.4f}")

# -----------------------------------------------------------------------------
# 5) Final SVM CV with learned kernel
# -----------------------------------------------------------------------------
K_final = np.array(build_kernel_matrix(params_k))
ker_scores = []

for tr, te in kf.split(K_final, y):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_final[np.ix_(tr, tr)], y[tr])
    preds = svc.predict(K_final[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y[te], preds))

print(f"\nQuantum-Metric Kernel CV Accuracy: {np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}")

# -----------------------------------------------------------------------------
# 6) Kernel heatmap visualization
# -----------------------------------------------------------------------------
plt.figure(figsize=(5, 5))
plt.title("Learned Kernel Matrix (|⟨ψ_i|ψ_j⟩|²)")
plt.imshow(K_final, cmap="viridis", origin="lower")
plt.colorbar(label="Kernel value")
plt.xlabel("Index j")
plt.ylabel("Index i")
plt.tight_layout()
plt.show()